### INE

In [617]:
%reset -sf
%clear

In [618]:
# Imports
import pandas as pd
import requests
import shutil
import time
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [619]:
# Sections txts
txts = ['01_Catalogo de Secciones.txt','02_Catalogo de Secciones.txt','03_Catalogo de Secciones.txt',
        '04_Catalogo de Secciones.txt','05_Catalogo de Secciones.txt','06_Catalogo de Secciones.txt',
        '07_Catalogo de Secciones.txt','08_Catalogo de Secciones.txt','09_Catalogo de Secciones.txt',
        '10_Catalogo de Secciones.txt','11_Catalogo de Secciones.txt','12_Catalogo de Secciones.txt',
        '13_Catalogo de Secciones.txt','14_Catalogo de Secciones.txt','15_Catalogo de Secciones.txt',
        '16_Catalogo de Secciones.txt','17_Catalogo de Secciones.txt','18_Catalogo de Secciones.txt',
        '19_Catalogo de Secciones.txt','20_Catalogo de Secciones.txt','21_Catalogo de Secciones.txt',
        '22_Catalogo de Secciones.txt','23_Catalogo de Secciones.txt','24_Catalogo de Secciones.txt',
        '25_Catalogo de Secciones.txt','26_Catalogo de Secciones.txt','27_Catalogo de Secciones.txt',
        '28_Catalogo de Secciones.txt','29_Catalogo de Secciones.txt','30_Catalogo de Secciones.txt',
        '31_Catalogo de Secciones.txt','32_Catalogo de Secciones.txt']

In [620]:
# Unifying TXTS
corpus = {}

for i in txts:
    print(i)
    corpus[i] = pd.read_csv('../Secciones/'+i, sep='|', encoding='latin-1')

01_Catalogo de Secciones.txt
02_Catalogo de Secciones.txt
03_Catalogo de Secciones.txt
04_Catalogo de Secciones.txt
05_Catalogo de Secciones.txt
06_Catalogo de Secciones.txt
07_Catalogo de Secciones.txt
08_Catalogo de Secciones.txt
09_Catalogo de Secciones.txt
10_Catalogo de Secciones.txt
11_Catalogo de Secciones.txt
12_Catalogo de Secciones.txt
13_Catalogo de Secciones.txt
14_Catalogo de Secciones.txt
15_Catalogo de Secciones.txt
16_Catalogo de Secciones.txt
17_Catalogo de Secciones.txt
18_Catalogo de Secciones.txt
19_Catalogo de Secciones.txt
20_Catalogo de Secciones.txt
21_Catalogo de Secciones.txt
22_Catalogo de Secciones.txt
23_Catalogo de Secciones.txt
24_Catalogo de Secciones.txt
25_Catalogo de Secciones.txt
26_Catalogo de Secciones.txt
27_Catalogo de Secciones.txt
28_Catalogo de Secciones.txt
29_Catalogo de Secciones.txt
30_Catalogo de Secciones.txt
31_Catalogo de Secciones.txt
32_Catalogo de Secciones.txt


In [621]:
# Appending
agg = pd.DataFrame()

for db in corpus.values():
    agg = agg.append(db)

In [622]:
# Sections as dict {Entidad:section_list}
seccions_dict = {}

for i in agg['ENTIDAD'].unique():
    seccions_dict[str(i)] = list(agg[agg['ENTIDAD']==i]['SECCION'])

In [623]:
# Open page

In [624]:
driver = webdriver.Chrome()

In [625]:
driver.get('https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/candidatura')

In [626]:
driver.maximize_window()

In [627]:
# Selectors 

In [628]:
def get_ent_sel():
    return Select(driver.find_element(By.ID, 'select-entidad_resultados'))
def get_sec_sel():
    return driver.find_element(By.ID, 'valueSeccion')
    
ent_selector = get_ent_sel()
sec_selector = get_sec_sel()

In [629]:
cols = ['Entidad','Seccion', 'Casilla','PAN', 'PRI', 'PRD', 'PVEM', 'PT', 'MC', 'MORENA','PAN/PRI/PRD','PAN/PRI','PAN/PRD','PRI/PRD',
        'PVEM/PT/MORENA','PVEM/PT','PVEM/MORENA','PT/MORENA','Candidaturas no registradas','Votos nulos','Total']

tables_agg_data = pd.DataFrame(columns=cols)

In [630]:
# Iterator
remove = ['Casilla', 'Acta digitalizada', 'Candidaturas no registradas', 'Votos nulos', 'Total', 'Observaciones en Acta',]

In [ ]:
tables_agg_data = tables_agg_data.drop_duplicates(subset=['Entidad','Seccion','Casilla'])

try:
    driver.switch_to.window(driver.window_handles[1])
    driver.close()
    driver.switch_to.window(driver.window_handles[0])
except:
    pass

driver.refresh()

ent_selector = get_ent_sel()
sec_selector = get_sec_sel()

sec_selector.send_keys(Keys.CONTROL + Keys.SHIFT + 'r')

try:
    index_ck = list(seccions_dict.keys()).index(current_k)
    for i in list(range(index_ck)):
        seccions_dict.pop(0)
    seccions_dict[current_k] = seccions_dict[current_k][current_s:]
except:
    pass

for k,v in seccions_dict.items():
    print(k)
    ent_selector.select_by_value(k) # Enter edo
    
    # For each section
    for s in tqdm(range(len(v))):
        
        # Reload
        #driver.refresh()
        #sec_selector = get_sec_sel()
        #sec_selector.send_keys(Keys.CONTROL + Keys.SHIFT + 'r')
        
        #ent_selector = get_ent_sel()
        #sec_selector = get_sec_sel()
        
        # Extract data
        current_k = k
        current_s = s
        print(seccions_dict[k][s])
        sec_selector.send_keys(str(seccions_dict[k][s]), Keys.ENTER) # Enter section
        sec_selector.send_keys(Keys.CONTROL + 'a') # Select section
        sec_selector.send_keys(Keys.DELETE) # Clear selection
        table_data = driver.find_element(By.CLASS_NAME, 'table').text # Table data
        table_data = table_data.split('\n') # Data treat
        
        for r in remove: # Remove not useful data
            table_data = [i for i in table_data if i != r]
        table_data = [i.split(' ') for i in table_data] # Convert to list
                
        for r in table_data:
            if 'Contigua' in r: # If contigua remove contigua number
                r.pop(2)
        for r in table_data:
            if 'Extraordinaria' in r: # If extra remove contigua number
                r.pop(2)
        for r in table_data:
            if 'Especial' in r: # If esp remove contigua number
                r.pop(2)
        for r in table_data:
            if 'Sin' in r: # If Sin remove
                with_sindex = [index for index, item in enumerate(r) if item == 'Sin']
                for i,j in zip(with_sindex,list(range(len(with_sindex)))):
                    r.pop(i-j)
                    
                with_dato_index = [index for index, item in enumerate(r) if item == 'dato']            
                for i in with_dato_index:
                    r[i] = 'sin dato'
        for r in table_data: # If last sin dato remove
            if r[-1]=='sin dato':
                r.pop()
        for r in table_data:
            if r[-1]=='vacíos':
                r.pop()
                r.pop()
                r.pop()
                r.pop()
        for r in table_data:
            if 'Ext' in r: # If extra remove
                r.pop(2)
                r.pop(3)
                r.pop(2)
                r[1] = 'Extraordinaria'
        for r in table_data:
            if 'Ilegible' in r: # If ilegible remove
                del r[2:]
                for _ in range(18):
                    r.append('Ilegible')
        for r in table_data:  
            if "-" in r: # Remove non counted yet
                table_data.remove(r)
                
        table_data = pd.DataFrame(table_data, columns=cols[1:])
        table_data.insert(0,'Entidad',k)
        
        if len([i for i in list(table_data['Casilla']) if i=="Contigua"]) > 0:
            my_list = list(table_data['Casilla'])
            contigua_counter = 1
            for index, item in enumerate(my_list):
                if item == "Contigua":
                    my_list[index] += f"_{contigua_counter}"
                    contigua_counter += 1        
            table_data['Casilla'] = my_list
            del my_list
            
        if len([i for i in list(table_data['Casilla']) if i=="Extraordinaria"]) > 0:
            my_list = list(table_data['Casilla'])
            extra_counter = 1
            for index, item in enumerate(my_list):
                if item == "Extraordinaria":
                    my_list[index] += f"_{extra_counter}"
                    extra_counter += 1        
            table_data['Casilla'] = my_list
            del my_list
            
        if len([i for i in list(table_data['Casilla']) if i=="Especial"]) > 0:
            my_list = list(table_data['Casilla'])
            extra_counter = 1
            for index, item in enumerate(my_list):
                if item == "Especial":
                    my_list[index] += f"_{extra_counter}"
                    extra_counter += 1        
            table_data['Casilla'] = my_list
            del my_list
        
        tables_agg_data = tables_agg_data.append(table_data)
        
        # Extract sabana        
        jpgs = driver.find_elements(By.XPATH,'//a[contains(@href, ".jpg")]')

        for i in range(len(jpgs)):
            try:
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((jpgs[i]))).click()
            except:
                time.sleep(1)
                WebDriverWait(driver, 10).until(EC.element_to_be_clickable((jpgs[i]))).click()
                print("ElementClickInterceptedException: Retrying")
            #finally:
            #    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((jpgs[i]))).click()
            #    print("ElementClickInterceptedException: Retrying 2")
                
            #driver.execute_script("arguments[0].scrollIntoView();", i)
            #time.sleep(2)
            #jpgs[i].click()
            driver.switch_to.window(driver.window_handles[1])
            file_url = driver.current_url

            response = requests.get(file_url, stream=True)
            if response.status_code == 200:
                with open('../Sabanas/'+str(k)+'_'+str(v[s])+'_'+str(i)+'.jpg', 'wb') as f:
                    response.raw.decode_content = True
                    shutil.copyfileobj(response.raw, f)
            driver.close()
            driver.switch_to.window(driver.window_handles[0])
            
        #v.pop(0)
        #print(f'Remaining in Entity: {len(v)}')